# Setup
## Import libraries

In [1]:
# Import the libraries and set the paths
import sys
import os
notebook_dir = os.getcwd()
root = os.path.abspath(os.path.join(notebook_dir, '../../'))
sys.path.append(root)
from IPython.display import Image, display

from utils.data.csv_parsing import load_csv_as_dicts, load_csv_as_dataframe
from utils.langchain.llm_model_selector import get_llm_from_model_name

import getpass
from pydantic import BaseModel
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run



## Load Data

In [2]:
article_path = '/data/transformed/FA-KES.csv'

articles = load_csv_as_dicts(root+article_path)
articles_df = load_csv_as_dataframe(root+article_path)

## Load API Keys

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("LANGSMITH_API_KEY")
_set_env("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "FakeNews Detection"

# Graph Definition

In [4]:
open_ai_model_name = 'gpt-4o-mini-2024-07-18'
anthropic_model_name = 'claude-3-haiku-20240307'

In [5]:
# main.py

from core.experiment import Experiment

# Define the prompt name
# prompt_name = "naive_zero_shot_classification"
# prompt_name = "robust_zero_shot_classification"
prompt_name = "few_shot_classification"


# Instantiate the experiment
experiment = Experiment(
    prompt_name=prompt_name,
    model_name=open_ai_model_name,
    dataset_name="FA-KES test",
    description="Evaluating graph-based misinformation detection system.",
    experiment_type="basic_classification"
)

# Run the evaluation
results = experiment.run_evaluation()

/Users/finlayduff/.pyenv/versions/3.10.4/envs/fakenews_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'basic_classification_few_shot_classification_gpt-4o-mini-2024-07-18_FA-KES test-980ee056' at:
https://smith.langchain.com/o/7ade50a2-3a1f-5106-9003-6a8cfb7b3652/datasets/ffa23208-7472-4ba3-b3c2-2efe90ff9c8f/compare?selectedSessions=d8eedf33-e863-4c46-8a5f-ef87a62ac4e7




100it [00:33,  3.00it/s]
